In [5]:
from twisted.internet import reactor
import json
from ctrader_fix import *
import datetime

In [6]:
# you can use two separate config files for QUOTE and TRADE
with open("config.json") as configFile:
    config = json.load(configFile)

In [7]:
client = Client(config["Host"], config["Port"], ssl = config["SSL"])

In [8]:
def send(request):
    diferred = client.send(request)
    diferred.addCallback(lambda _: print("\nSent: ", request.getMessage(client.getMessageSequenceNumber()).replace("", "|")))

In [9]:
def onMessageReceived(client, responseMessage): # Callback for receiving all messages
    print("\nReceived: ", responseMessage.getMessage().replace("", "|"))
    # We get the message type field value
    messageType = responseMessage.getFieldValue(35)
    # we send a security list request after we received logon message response
    if messageType == "A":
        securityListRequest = SecurityListRequest(config)
        securityListRequest.SecurityReqID = "A"
        securityListRequest.SecurityListRequestType = 0
        send(securityListRequest)
    # After receiving the security list we send a market order request by using the security list first symbol
    elif messageType == "y":
        # We use getFieldValue to get all symbol IDs, it will return a list in this case
        # because the symbol ID field is repetitive
        symboldIds = responseMessage.getFieldValue(55)
        if config["TargetSubID"] == "TRADE":
            newOrderSingle = NewOrderSingle(config)
            newOrderSingle.ClOrdID = "B"
            newOrderSingle.Symbol = symboldIds[1]
            newOrderSingle.Side = 1
            newOrderSingle.OrderQty = 1000
            newOrderSingle.OrdType = 1
            newOrderSingle.Designation = "From Jupyter"
            send(newOrderSingle)
        else:
            marketDataRequest = MarketDataRequest(config)
            marketDataRequest.MDReqID = "a"
            marketDataRequest.SubscriptionRequestType = 1
            marketDataRequest.MarketDepth = 1
            marketDataRequest.NoMDEntryTypes = 1
            marketDataRequest.MDEntryType = 0
            marketDataRequest.NoRelatedSym = 1
            marketDataRequest.Symbol = symboldIds[1]
            send(marketDataRequest)
    # after receiving the new order request response we stop the reactor
    # And we will be disconnected from API
    elif messageType == "8" or messageType == "j":
        print("We are done, stopping the reactor")
        reactor.stop()

def disconnected(client, reason): # Callback for client disconnection
    print("\nDisconnected, reason: ", reason)

def connected(client): # Callback for client connection
    print("Connected")
    logonRequest = LogonRequest(config)
    send(logonRequest)

# Setting client callbacks
client.setConnectedCallback(connected)
client.setDisconnectedCallback(disconnected)
client.setMessageReceivedCallback(onMessageReceived)

In [10]:
# Starting the client service
client.startService()
# Run Twisted reactor, we imported it earlier
reactor.run()

TypeError: code() argument 13 must be str, not int